# Assignment 3
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. All questions are worth the same number of points except question 1 which is worth 17% of the assignment grade.

**Note**: Questions 2-13 rely on your question 1 answer.

In [1]:
import pandas as pd
import numpy as np

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`. Additionally, there are several countries with Numeric digits in their name. Make sure to remove these as well, e.g. `'Italy9'` should be `'Italy'`. 

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [77]:

import pandas as pd
import numpy as np

def answer_one():
    # Imported the file 
    Energy = pd.read_excel('C:/Users/HP/Downloads/Files/assignments/course1_assignment3/assets/Energy Indicators.xls',
                           skiprows = 17, skipfooter = 38) 
    
    # Removed the unneccery data
    Energy.drop(['Unnamed: 0','Unnamed: 1'] , axis = 1, inplace = True)
    Energy = Energy.dropna()
    
    # Renamed the columns 
    #['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]
    Energy = Energy.rename(columns={'Unnamed: 2': 'Country', 'Petajoules': 'Energy Supply'
                                    ,'Gigajoules': 'Energy Supply per Capita','%': '% Renewable' })
    
    # Convert the peta joules into giga joules 1,000,000
    Energy['Energy Supply'] = Energy['Energy Supply'] * 1000000
    
    # convert all the ... with np.NaN
    Energy.replace('...',np.NaN , inplace = True)
     # To remove the spaces in the names and the numbers in the name of type : r"^(Bolivia).*":"Bolivia"
    # We have used the str.replace cause we wanted to replace and the replace method is of str object
    Energy['Country'] = Energy['Country'].str.replace(r'\s\(.*\)', "")
    Energy['Country'] = Energy['Country'].str.replace(r'\d*', "")
    
    # replace the names "Republic of Korea": "South Korea", "United States of America": "United States",
    #"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    #"China, Hong Kong Special Administrative Region": "Hong Kong"
    Energy= Energy.replace({"Country":{r"^(Republic of Korea).*": "South Korea",
                               r'^(United States of America).*':'United States',
                               r"^(United Kingdom of Great Britain and Northern Ireland).*": "United Kingdom",
                               r"^(China, Hong Kong Special Administrative Region).*": "Hong Kong"
                              }},regex=True)
    # Just a check if the countries name have actually changed
    #country =Energy[ Energy['Country'] == "United Kingdom"]
    # import the GDP file 
    GDP = pd.read_csv('C:/Users/HP/Downloads/Files/assignments/course1_assignment3/assets/world_bank.csv'
                     ,skiprows = 4)
    
    # Replace the names "Korea, Rep.": "South Korea", 
    #"Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"
    GDP['Country Name'] = GDP['Country Name'].replace({
        "Korea, Rep.": "South Korea",
        "Iran, Islamic Rep.": "Iran",
        "Hong Kong SAR, China": "Hong Kong"
    })
    country = GDP[ GDP['Country Name'] == "Korea, Rep."]
    GDP.rename(columns = {'Country Name': 'Country'}, inplace = True)
    
    # Import the file scimagojr-3
    ScimEn = pd.read_excel('C:/Users/HP/Downloads/Files/assignments/course1_assignment3/assets/scimagojr-3.xlsx')
    
    # Only the years from 2006 to 2015
    years = [str(year) for year in range(2006, 2016)]
    GDP = GDP[['Country'] + years]
    
    # Join the three data sets 
    combined_1 = pd.merge(GDP, Energy , how = 'inner' , on = 'Country')
    combined_2 = pd.merge(ScimEn, combined_1, on = 'Country', how='left')
    
    combined_2.set_index('Country', inplace = True)
    combined_2 = combined_2[combined_2['Rank'] <= 15]
    combined_2.sort_values(by = 'Rank' , inplace = True)
    
    return combined_2

answer_one()

#Finally, load the Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology
#from the file assets/scimagojr-3.xlsx, which ranks countries based on their journal contributions 
#in the aforementioned area. Call this DataFrame ScimEn.

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,Energy Supply,Energy Supply per Capita,% Renewable
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
United States,2,96661,94747,792274,265436,8.20,230,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13,90838000000,286.0,11.570980
Japan,3,30504,30287,223024,61554,7.31,134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
United Kingdom,4,20944,20357,206091,37874,9.84,139,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12,7920000000,124.0,10.600470
Russian Federation,5,18534,18301,34266,12422,1.85,57,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12,30709000000,214.0,17.288680
Canada,6,17899,17620,215003,40930,12.01,149,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12,10431000000,296.0,61.945430
Germany,7,17027,16831,140566,27426,8.26,126,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12,13261000000,165.0,17.901530
India,8,15005,14841,128763,37209,8.58,115,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12,33195000000,26.0,14.969080
France,9,13153,12973,130632,28601,9.93,114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"

assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"


In [4]:
# Cell for autograder.


### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [5]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [105]:
energy = pd.read_excel('assets/Energy Indicators.xls', skiprows=17)

# remove header and footer
energy = energy[:227]

# drop useless columns
energy = energy.drop(energy.columns[[0, 1]], axis=1)

# rename columns
energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

# replace ... with NaN
energy.replace('...', np.nan, inplace = True)

# convert Energy Supply units
energy['Energy Supply'] = energy['Energy Supply'] * 1000000

# remove all numbers from country names and parenthesis
def remove_numbers(country):
    return ''.join(filter(lambda x: not x.isdigit(), country))

def remove_parenthesis(country):
    start = country.find('(')
    if start > -1:
        return country[:start-1]
    return country

energy['Country'] = energy['Country'].apply(remove_numbers)
energy['Country'] = energy['Country'].apply(remove_parenthesis)

# replace country names
country_replace = {'Republic of Korea': 'South Korea',
                   'United States of America': 'United States',
                   'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
                   'China, Hong Kong Special Administrative Region': 'Hong Kong',
                  }
# energy.replace(country_replace, inplace = True)
energy.replace({'Country': country_replace}, inplace = True)

# energy[energy['Country'] == 'Iran']
GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)

# rename Country Name to Country so that merge will work later on.
GDP.rename(columns={'Country Name': 'Country'}, inplace = True)

country_replace = {'Korea, Rep.': 'South Korea', 
                   'Iran, Islamic Rep.': 'Iran',
                   'Hong Kong SAR, China': 'Hong Kong'
                  }

GDP.replace({'Country': country_replace}, inplace = True)

# GDP[GDP['Country'] == 'Iran']
ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
def answer_two():
    global energy, GDP, ScimEn

    un = pd.merge(pd.merge(energy, GDP, on = 'Country', how = 'outer'), ScimEn, on = 'Country', how = 'outer')
    int = pd.merge(pd.merge(energy, GDP, on = 'Country'), ScimEn, on = 'Country')
    
    return (len(un) - len(int))

answer_two()

158

In [106]:
assert type(answer_two()) == int, "Q2: You should return an int number!"


### Question 3
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [120]:
def answer_three():
    Top15 = answer_one()
    avgGDP = Top15[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].mean(axis = 1).sort_values(ascending = False)
    
    return avgGDP

#answer_three()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

In [9]:
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"


### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [10]:
def answer_four():
    data = answer_three()
    sixth = data.index[5]
    data2 = answer_one()
    
    # We can not access the data of united state as the country column is index
    
    data2 = data2.reset_index()
    data2 = data2[data2['Country'] == sixth]
    
    data3 = data2['2015'] - data2['2006']
    
    return data3

    raise NotImplementedError()
    
answer_four()

3    2.467027e+11
dtype: float64

In [11]:
# Cell for autograder.


### Question 5
What is the mean energy supply per capita?

*This function should return a single number.*

In [118]:
def answer_five():
    Top15 = answer_one()
    
    return Top15['Energy Supply per Capita'].mean()

answer_five()

157.6

In [13]:
# Cell for autograder.


### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [14]:
def answer_six():
    data = answer_one()
    data = data.reset_index()
    #% Renewable
    index = data['% Renewable'].idxmax()
    
    percentage = data.iloc[index]['% Renewable']
    country = data.iloc[index]['Country']
    
    return (country , percentage)
    raise NotImplementedError()
answer_six()

('Brazil', 69.64803)

In [15]:
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"

assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"


### Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [16]:
def answer_seven():
    data = answer_one()
    # We reset the index 
    data = data.reset_index()
    
    # to calculate the ratio of citations 
    data['citation_ratio'] = data['Self-citations'] / data['Citations']
    
    # the index of highest citation_ratio
    maximum = data['citation_ratio'].idxmax()
    
    country = data.iloc[maximum]['Country']
    value = data.iloc[maximum]['citation_ratio']
    
    return (country, value)
    raise NotImplementedError()
answer_seven()

('China', 0.6893126179389422)

In [17]:
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"

assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"


### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

In [18]:
def answer_eight():
    data = answer_one()
    data = data.reset_index()
    
    # Estimate the population by the ratio of Energy supply and Energy Supply per Capita
    data['Estimated Population'] = data['Energy Supply']/data['Energy Supply per Capita']
    
    data = data.sort_values('Estimated Population', ascending = False)
    name = data.iloc[2]['Country']
    return name
    raise NotImplementedError()
answer_eight()

'United States'

In [19]:
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"


### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [121]:
def answer_nine():
    data = answer_one()
    data = data.reset_index()
    data['Estimated Population'] = data['Energy Supply']/data['Energy Supply per Capita']
    
    # column that estimates the number of citable documents per person
    # "Energy Supply per Capita" means the energy supply available for each individual in a country.
    data['Citable per person'] = data['Citable documents']/data['Estimated Population']
    
    data[['Citable per person', 'Energy Supply per Capita']] = data[['Citable per person', 'Energy Supply per Capita']].apply(pd.to_numeric, errors='coerce')
    
    data_cleaned = data.dropna(subset = ['Citable per person','Energy Supply per Capita'])
    
    data_corr = data_cleaned['Citable per person'].corr(data_cleaned['Energy Supply per Capita'])
    
    return data_corr
    raise NotImplementedError()
answer_nine()
#plot(()

0.7940010435442946

In [122]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [123]:
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"


### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [43]:
def answer_ten():
    data = answer_one()
    # To calculate the median of % Renewable 
    median = data['% Renewable'].median()
    # As true is 1 and false is 0 we just did boolean masking and converted true and false into their numerical eqvalents
    data['HighRenew'] = (data['% Renewable']>= median).astype(int)  
    data = data.sort_values('Rank', ascending = True) 
    
    return data['HighRenew']

    raise NotImplementedError()
answer_ten()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

In [44]:
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"


### Question 11
Use the following dictionary to group the Countries by Continent, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [51]:
def answer_eleven():
    data = answer_one()
    data = data.reset_index()
    
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    data['Continents'] = data['Country'].map(ContinentDict)
    
    # Estimated population
    data['Estimated Population'] = data['Energy Supply']/data['Energy Supply per Capita']
    
    df = data.groupby('Continents')['Estimated Population'].agg(['size', 'sum', 'mean', 'std'])
    
    return df

    raise NotImplementedError()
    
answer_eleven()

,size,sum,mean,std
Continents,,,,
Asia,5,2898666386.6106,5.797333e+08,6.790979e+08
Australia,1,23316017.316017,2.331602e+07,NaN
Europe,6,457929667.216372,7.632161e+07,3.464767e+07
North America,2,352855249.48025,1.764276e+08,1.996696e+08
South America,1,205915254.237288,2.059153e+08,NaN


In [52]:
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"

assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"

assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"


### Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [95]:
def answer_twelve():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top15["bins"] = pd.cut(Top15["% Renewable"], 5)
    
    return Top15.groupby([ContinentDict, Top15['bins']]).size()
    raise NotImplementedError()
answer_twelve()

Country        bins            
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Australia      (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
               (42.701, 56.174]    0
               (56.174, 69.648]    0
North America  (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
South America  (2.212, 15.753]     0
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
dtype: int64

In [96]:
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"

assert len(answer_twelve()) == 25, "Q12: Wrong result numbers!"


### Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a series `PopEst` whose index is the country name and whose values are the population estimate string*

In [107]:
def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    
    # https://stackoverflow.com/questions/22617/format-numbers-to-strings-in-python
    return Top15['PopEst'].apply(lambda str: '{0:,}'.format(str))

answer_thirteen()

Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: PopEst, dtype: object

In [108]:
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"

assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [78]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")